In [3]:
import os
import re
import json
from sklearn.feature_extraction.text import CountVectorizer

In [134]:
vectorizer = CountVectorizer()

In [149]:
def frequencyDocument(json_dict, file_index):
    for words in json_dict.values():
        if words in ['', '--', ' ', '.', ' -']:
            continue
        words = words.strip().split(' ')
        try:
            vectorizer.fit_transform(words)
        except:
            #print(words)
            pass
    for word, frequency in vectorizer.vocabulary_.items():
        freqDoc = (file_index, frequency)
        if not inv_index_frequency.get(word, False):
            inv_index_frequency[word] = sef(freqDoc)
        else:
            inv_index_frequency[word].add(freqDoc)

In [150]:
def twoTermsDocument(json_dict, file_index):
    for key, words in json_dict.items():
        if words in ['', '--', ' ', '.', ' -']:
            continue
        words = words.strip().split(' ')
        for word in words:
            newKey = word + '.' + key
            if not inv_index_twoTermsDocs.get(newKey, False):
                inv_index_twoTermsDocs[newKey] = set(file_index)
            else:
                inv_index_twoTermsDocs[newKey].add(file_index)

In [151]:
inv_index_frequency = {}
inv_index_twoTermsDocs = {}
path_db = os.path.abspath('./db')
for file_name in os.listdir(path_db):
    file_path = path_db + '\\' + file_name
    with open(file_path, 'r', encoding='utf-8') as file:
        file_text = file.read()
        file_text = re.sub(r'\\xc2|\\xae|\\xe2|\\x80|\\x99|\\x84|\\xa2|\\x93|\\xa0|\\', '', file_text)
        json_dict = json.loads(file_text[:-1])
    frequencyDocument(json_dict, file_name)
    twoTermsDocument(json_dict, file_name)

In [143]:
inv_index_frequency

{'introducing': [('1', 20), ('421', 23), ('761', 14)],
 'assassins': [('1', 4)],
 'creed': [('1', 9), ('670', 9)],
 'rogue': [('1', 29), ('343', 22), ('62', 17), ('683', 19)],
 'the': [('1', 33),
  ('100', 8),
  ('1001', 20),
  ('1002', 29),
  ('103', 12),
  ('104', 30),
  ('105', 30),
  ('106', 13),
  ('107', 13),
  ('109', 19),
  ('111', 6),
  ('112', 6),
  ('113', 6),
  ('114', 6),
  ('119', 15),
  ('120', 26),
  ('125', 17),
  ('127', 16),
  ('131', 29),
  ('132', 29),
  ('147', 17),
  ('148', 17),
  ('150', 21),
  ('151', 34),
  ('152', 29),
  ('153', 36),
  ('154', 33),
  ('155', 33),
  ('156', 33),
  ('157', 33),
  ('158', 30),
  ('159', 30),
  ('16', 21),
  ('169', 3),
  ('170', 18),
  ('174', 27),
  ('175', 27),
  ('177', 26),
  ('178', 35),
  ('179', 9),
  ('180', 24),
  ('181', 24),
  ('184', 15),
  ('185', 32),
  ('186', 20),
  ('187', 20),
  ('188', 37),
  ('189', 15),
  ('191', 9),
  ('192', 34),
  ('193', 19),
  ('194', 19),
  ('201', 33),
  ('204', 20),
  ('206', 27),
 

In [152]:
inv_index_twoTermsDocs

{'Assassins.Game': {'1'},
 'Creed.Game': {'1', '115', '226', '245', '337', '448'},
 'Rogue.Game': {'1', '343', '814'},
 '197,96.Price': {'1'},
 'windows.OS': {'1',
  '10',
  '100',
  '1000',
  '1001',
  '1002',
  '101',
  '102',
  '103',
  '104',
  '106',
  '108',
  '109',
  '11',
  '110',
  '111',
  '115',
  '116',
  '119',
  '12',
  '120',
  '121',
  '122',
  '123',
  '124',
  '125',
  '126',
  '127',
  '128',
  '129',
  '13',
  '130',
  '131',
  '135',
  '136',
  '138',
  '139',
  '14',
  '140',
  '141',
  '142',
  '147',
  '149',
  '15',
  '152',
  '158',
  '160',
  '169',
  '17',
  '171',
  '172',
  '173',
  '174',
  '175',
  '176',
  '177',
  '178',
  '179',
  '18',
  '184',
  '185',
  '186',
  '189',
  '19',
  '191',
  '193',
  '195',
  '197',
  '198',
  '199',
  '20',
  '201',
  '202',
  '203',
  '204',
  '205',
  '206',
  '207',
  '21',
  '212',
  '213',
  '214',
  '215',
  '216',
  '217',
  '218',
  '219',
  '22',
  '220',
  '221',
  '222',
  '223',
  '224',
  '225',
  '226',